# Librerías utilizadas

In [10]:
%pylab
#pylab.rcParams['figure.figsize'] = (10, 6)

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


# Funciones

In [57]:
def GaussianNoise(SNR,Ps,M,N):
    """
    Recibe una potencia de señal Ps y una relación señal-ruido deseada
    SNR y retorna un vector Nu correspondiente a muestras del ruido 
    blanco gaussiano a sumarle a la señal
    """
    mean_W = 0
    var_W = Ps/(10**(SNR/10))
    std_W = np.sqrt(var_W)
    W = np.asmatrix(np.random.normal(mean_W, std_W, size=(M,N)))
    return (W)

def print_matrix(list_of_list):
    number_width = len(str(max([max(i) for i in list_of_list])))
    cols = max(map(len, list_of_list))
    output = '+'+('-'*(number_width+2)+'+')*cols + '\n'
    for row in list_of_list:
        for column in row:
            output += '|' + ' {:^{width}d} '.format(column, width = number_width)
        output+='|\n+'+('-'*(number_width+2)+'+')*cols + '\n'
    return output

def doa_rootmusic_samples(Fc,s_amp,s_freq,t,t0,d,theta,phi,Mx,My,D,SNR):
    M = Mx * My
    # Definición de señales y muestras
    # Genero la matriz F de tamaño (D,N)
    lambda_c = c / Fc
    K = 2 * np.pi / (lambda_c)
    F = np.asmatrix(np.empty((D, N)))
    for i in range(0, D):
        F[i,:] = s_amp[i] * np.cos(2 * np.pi * s_freq[i] * (t+t0))

    # Genero la matriz A de tamaño (M,D) considerando $g(\theta)=1 \quad \forall \  \theta$
    A = np.asmatrix(np.empty((M, D), dtype='complex'))

    for i in range(0, Mx):
        for j in range(0,My):
            for k in range(0, D):
                A[Mx*j+i, k] = np.e ** (-1j * (i * K[k] * d * sin(theta[k]) * cos(phi[k]) + j * K[k] * d * sin(theta[k]) * sin(phi[k])))

    # Genero el vector W de tamaño (M,1,N) de ruido $W\sim N(0,1)$
    Ps = 0
    for i in range(0,D):
        Ps = Ps + (s_amp[i]**2)/2
    W = GaussianNoise(SNR,Ps,M,N)

    # Genero el vector de muestras $X=A \times F + W$ de tamaño (M,1,N)
    X = np.asmatrix(np.empty((M,N),dtype=complex))
    X_matrix = np.asmatrix(np.empty((M,1)),dtype=complex)
    for n in range (0,N):
        F_matrix = F[:,n]
        W_matrix = W[:,n]
        X_matrix = A @ F_matrix + W_matrix
        X[:, n] = X_matrix

    return X

#def doa_rootmusic_estimation(X, K_est):
def EN_rootmusic(X):
    M = X.shape[0]
    N = X.shape[1]

    #%% Calculo el vector de covarianza S
    S = np.asmatrix(np.empty((M,M),dtype=complex))
    for n in range (0,N):
        X_matrix=np.asmatrix(X[:,n])
        S = S + (1/N)*(X_matrix @ X_matrix.H)

    # Encuentro autovalores y autovectores S y el autovalor mínimo $\lambda_{min}$ tal que $|S-\lambda_{min}\cdot S_0|=0$
    [aval, avec] = eig(S)
    S0 = np.identity(M)
    p = aval.argsort()
    aval=np.abs(aval[p])
    avec=avec[:,p]
    aval_min = aval[0]

    # Encuentro la multiplicidad Q de $\lambda_min$ y el número estimado de señales $\hat{D}$
    bins_aval=histogram(aval,bins=50)
    for i in range(0,aval.size):
        if bins_aval[0][i] == 0:
            umbral = bins_aval[1][i]
            break

    Q = 0

    for i in range (0,M):
        if aval[i]<umbral:
            Q += 1
    D_est = M - Q

    # Formo la matriz de subespacio de ruido $E_N$
    EN = np.asmatrix(avec[:,0:Q])

    # Calculo la matriz $C = E_N E_N^H$
    #C = EN @ EN.H

    # Encuentro el polinomio Root-MUSIC
    #Pz = np.zeros(2*M-1,dtype=complex)

    #for i in range (0,Pz.size):
    #    Pz[i]=sum(diag(C,k=-M+1+i))

    # Encuentro las raíces del polinomio y las DOA
    #Pz_roots = roots(Pz)
    #Pz_roots = Pz_roots[(abs(Pz_roots)<=1)]
    #Pz_roots = Pz_roots[argsort(abs(Pz_roots))]
    #Pz_roots = Pz_roots[-D_est:]
    #theta_est = arccos( angle(Pz_roots) /(K_est[0]*d) )*180/pi

    #return theta_est
    return EN,D_est

# Definición de constantes y unidades

In [58]:
kHz = 10 ** 3
MHz = 10 ** 6
GHz = 10 ** 9
ms = 10 ** -3
c = 3 * 10 ** 8
km = 10 ** 3

In [162]:
Mx = 4  # Número de sensores en la dirección x
My = 4  # Número de sensores en la dirección y
D = 1  # Número de señales
N = 100 # Número de snapshots

fs = 64 * MHz

T = 5 * ms # Tiempo de toma de muestras
t0 = 0

theta_deg = np.array([80])  #DOA en grados
phi_deg = np.array([0])
theta = theta_deg * np.pi / 180           #DOA en radianes
phi = phi_deg * np.pi / 180           #DOA en radianes

# Defición de señal y array de sensores
Fc = np.array([436*MHz,436*MHz,436*MHz,436*MHz]) # Frecuencia de portadora de la señal transmitida 1
lambda_c = c / Fc
d = lambda_c[0] / 2  #Separación entre sensores

s_amp = np.array([100,100,100,100])
s_freq = np.array([440,3000,40000,10000])

SNR = 7

t = np.random.randint(0, int(T * fs), size=N) * 1 / fs  # Vector de tiempos de muestreo (tomas de snapshots)

K_est = 2 * np.pi * Fc/c
theta_est = np.linspace(0, pi/2 ,num=50)
phi_est = np.linspace(-pi, pi ,num=50)

In [163]:
X = doa_rootmusic_samples(Fc, s_amp, s_freq, t,0, d, theta, phi, Mx, My, D, SNR)

In [196]:
EN0, D_est0 = EN_rootmusic(X[0:4])
EN1, D_est1 = EN_rootmusic(X[4:8])
EN2, D_est2 = EN_rootmusic(X[8:12])
EN3, D_est3 = EN_rootmusic(X[12:16])

#EN = concatenate((EN0,EN1,EN2,EN3),axis=1)
#EN = EN0
#EN = (EN0 + EN1 + EN2 + EN3)/4
EN, D_est = EN_rootmusic(X)

In [195]:
# Calculo la matriz $C = E_N E_N^H$
C = EN @ EN.H

# Encuentro el polinomio Root-MUSIC
Pz = np.zeros(2*M-1,dtype=complex)

for i in range (0,Pz.size):
    Pz[i]=sum(diag(C,k=-M+1+i))

# Encuentro las raíces del polinomio y las DOA
Pz_roots = roots(Pz)
Pz_roots = Pz_roots[(abs(Pz_roots)<=1)]
Pz_roots = Pz_roots[argsort(abs(Pz_roots))]
Pz_roots = Pz_roots[-1:]
theta_est = arccos( angle(Pz_roots) /(K_est[0]*d) )*180/pi

In [159]:
abs(Pz_roots)

array([0.99206105])

In [160]:
10 - theta_est

array([-0.00402348])

In [161]:
10 - theta_est0

array([0.00460622])